In [2]:
import pandas as pd
import numpy as np
import requests
import json

# Google API Key
#from config import gkey

In [5]:
boston2019_df = pd.read_csv("Resources/2019boston_marathon_runners.csv")
boston2019_df.head()

,BibNumber,FullName,SortName,AgeOnRaceDay,Gender,City,StateAbbrev,StateName,Zip,CountryOfResAbbrev,CountryOfResName,CountryOfCtzAbbrev,CountryOfCtzName,OfficialTime,RankOverall,RankOverGender,RankOverDivision,EventGroup,SubGroupLabel,SubGroup
0,2,Lawrence Cherono,"Cherono, Lawrence",30,M,Eldoret,NaN,NaN,NaN,KEN,Kenya,KEN,Kenya,2:07:57,1,1,1,Runners,NaN,NaN
1,6,Lelisa Desisa,"Desisa, Lelisa",29,M,Ambo,NaN,NaN,NaN,ETH,Ethiopia,ETH,Ethiopia,2:07:59,2,2,2,Runners,NaN,NaN
2,7,Kenneth Kipkemoi,"Kipkemoi, Kenneth",34,M,Eldoret,NaN,NaN,NaN,KEN,Kenya,KEN,Kenya,2:08:07,3,3,3,Runners,NaN,NaN
3,8,Felix Kandie,"Kandie, Felix",32,M,Iten,NaN,NaN,NaN,KEN,Kenya,KEN,Kenya,2:08:54,4,4,4,Runners,NaN,NaN
4,11,Geoffrey Kirui,"Kirui, Geoffrey",26,M,Keringet,NaN,NaN,NaN,KEN,Kenya,KEN,Kenya,2:08:55,5,5,5,Runners,NaN,NaN
